In [ ]:
%load_ext autoreload
%autoreload 2

import sys, os, json
sys.path.append("../")
import model_trainer

# Overfitting a Toy Model Using model_trainer

__Author:__ Sebastian Wagner-Carena

__Created:__ 10/24/2019

__Last Run:__ 10/24/2019

__Goals:__ Learn how to use model_trainer to overfit the three types of models used by ovejero

__Before running this notebook:__ Run the Generate_Config notebook to understand what goes into the configuration files for overjero

We'll start by loading up the test configuration file made by Generate_Config and inspecting it.

In [ ]:
json_path = os.getcwd()[:-5]+'tests/test_data/' + 'test.json'
with open(json_path,'r') as json_f:
    cfg = json.load(json_f)
print(cfg)

A lot of good information there! This is a good config file to start with. Let's go ahead and change a few paths and use it for our toy model.

In [ ]:
# Change the model weights to point to the demo directory! Same for log file.
print('old path:')
print(cfg['training_params']['model_weights'])

cfg['training_params']['model_weights'] = os.getcwd() + '/test_model.h5'
cfg['training_params']['tensorboard_log_dir'] = os.getcwd() + '/test_logs.txt'
print('new path:')
print(cfg['training_params']['model_weights'])

# Also let's start with the easy diagonal case
cfg['training_params']['bnn_type'] = 'diag'

# Now let's go ahead and save this as our new configuration file
diag_json_path = os.getcwd() + '/diag.json'
with open(diag_json_path,'w') as json_f:
    json.dump(cfg,json_f,indent=4)

All we have to do is call the main function of model_trainer with the path to our config file! You should see the loss go down as the model learns to overfit to the lenses in the very small training set. Because the random seed is set by the configuration file the final loss should be 2.9391

In [ ]:
# This is equivalent to 'python -m model_trainer diag_json_path' in the terminal where diag_json_path is the path.
sys.argv = ['model_trainer',diag_json_path]
model_trainer.main()

In [ ]:
# Clean up the files that were created by this notebook in the test directory.
test_data_path = os.getcwd()[:-5]+'tests/test_data/'
os.remove(test_data_path+'new_metadata.csv')
os.remove(test_data_path+'norms.npy')
os.remove(test_data_path+'tf_record_test')
os.remove('test_model.h5')
os.remove('diag.json')